In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
import joblib

# Завантажуємо денні дані та створюємо ознаки
df = pd.read_csv('../data/dataset_for_modeling.csv', index_col='DateTime', parse_dates=True)
df_daily = df['Global_active_power'].resample('D').sum().to_frame()
df_daily['day_of_week'] = df_daily.index.dayofweek
df_daily['month'] = df_daily.index.month
df_daily['day_of_year'] = df_daily.index.dayofyear

X = df_daily.drop(columns=['Global_active_power'])
y = df_daily['Global_active_power']

print("Дані для підбору гіперпараметрів готові.")

Дані для підбору гіперпараметрів готові.


In [2]:
# 1. Визначаємо сітку параметрів, які хочемо перевірити
# У реальному проєкті тут буде набагато більше значень
param_grid = {
    'n_estimators': [100, 300],
    'max_depth': [3, 5],
    'learning_rate': [0.05, 0.1]
}

# 2. Ініціалізуємо модель
model = xgb.XGBRegressor(random_state=42)

# 3. Ініціалізуємо TimeSeriesSplit для використання всередині GridSearchCV
tscv = TimeSeriesSplit(n_splits=3)

# 4. Створюємо об'єкт GridSearchCV
# scoring='neg_mean_absolute_error' - ми шукаємо мінімальну помилку, тому використовуємо негативне значення
grid_search = GridSearchCV(estimator=model,
                           param_grid=param_grid,
                           scoring='neg_mean_absolute_error',
                           n_jobs=-1,
                           cv=tscv,
                           verbose=2) # verbose=2 буде показувати детальний прогрес

print("GridSearchCV налаштовано. Починаємо пошук...")

GridSearchCV налаштовано. Починаємо пошук...


In [3]:
# Запускаємо пошук найкращих параметрів
grid_search.fit(X, y)

print("\n--- Результати підбору гіперпараметрів ---")
print(f"Найкращі параметри: {grid_search.best_params_}")
# Результат grid_search.best_score_ буде від'ємним, тому беремо модуль
print(f"Найкращий MAE (на крос-валідації): {-grid_search.best_score_:.2f}")

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] END ..learning_rate=0.05, max_depth=3, n_estimators=100; total time=   0.0s
[CV] END ..learning_rate=0.05, max_depth=3, n_estimators=100; total time=   0.0s
[CV] END ..learning_rate=0.05, max_depth=3, n_estimators=300; total time=   0.0s
[CV] END ..learning_rate=0.05, max_depth=3, n_estimators=100; total time=   0.0s
[CV] END ..learning_rate=0.05, max_depth=5, n_estimators=100; total time=   0.0s
[CV] END ..learning_rate=0.05, max_depth=5, n_estimators=100; total time=   0.0s
[CV] END ..learning_rate=0.05, max_depth=5, n_estimators=300; total time=   0.0s
[CV] END ...learning_rate=0.1, max_depth=3, n_estimators=100; total time=   0.0s
[CV] END ..learning_rate=0.05, max_depth=5, n_estimators=100; total time=   0.0s
[CV] END ...learning_rate=0.1, max_depth=3, n_estimators=100; total time=   0.0s
[CV] END ...learning_rate=0.1, max_depth=3, n_estimators=100; total time=   0.0s
[CV] END ..learning_rate=0.05, max_depth=3, n_est

In [4]:
# Найкраща модель доступна в атрибуті best_estimator_
best_model = grid_search.best_estimator_

# Зберігаємо її
joblib.dump(best_model, '../models/xgboost_tuned_model.pkl')

print("\nНайкраща, натюнена модель XGBoost успішно збережена.")


Найкраща, натюнена модель XGBoost успішно збережена.
